In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf


conf = SparkConf() \
        .setAppName("CLM") \
        .setMaster("local[2]") \
        .set("spark.jars", "file:///home/hiep/Downloads/postgresql-42.7.0.jar") \
        .set("spark.driver.extraClassPath", "postgresql-42.7.0.jar") \
        .set("spark.repl.local.jars", "	file:///home/hiep/Desktop/postgresql-42.7.0.jar")
        
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Tạo SparkSession
# spark = SparkSession.builder \
#     .appName("PostgresSparkConnect") \
#     .master("local[2]") \
#     .config("spark.jar.packages", "/home/hiep/Downloads/postgresql-42.7.0.jar") \
#     .config("spark.driver.extraClassPath", "/home/hiep/Downloads/postgresql-42.7.0.jar") \
#     .config("spark.executor.extraClassPath", "/home/hiep/Downloads/postgresql-42.7.0.jar") \
#     .getOrCreate()

24/07/03 16:50:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/03 16:50:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark

In [57]:
spark.stop()

In [4]:


df_customers = spark.read \
                    .format("jdbc") \
                    .option("driver","org.postgresql.Driver") \
                    .option("url","jdbc:postgresql://localhost:5432/dbt") \
                    .option("dbtable","raw_layer.customers") \
                    .option("user","dbt") \
                    .option("password","dbt") \
                    .load()
                    
df_customers.show(10)

24/07/03 16:50:16 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+-----------+-------+--------------------+----------+-------------------+-------------------+
|customer_id|zipcode|                city|state_code|   datetime_created|   datetime_updated|
+-----------+-------+--------------------+----------+-------------------+-------------------+
|          1|  14409|              franca|        SP|2017-10-18 00:00:00|2017-10-18 00:00:00|
|          2|  09790|sao bernardo do c...|        SP|2017-10-18 00:00:00|2017-10-18 00:00:00|
|          3|  01151|           sao paulo|        SP|2017-10-18 00:00:00|2017-10-18 00:00:00|
|          4|  08775|     mogi das cruzes|        SP|2017-10-18 00:00:00|2017-10-18 00:00:00|
|          5|  13056|            campinas|        SP|2017-10-18 00:00:00|2017-10-18 00:00:00|
|          6|  89254|      jaragua do sul|        SC|2017-10-18 00:00:00|2017-10-18 00:00:00|
|          7|  04534|           sao paulo|        SP|2017-10-18 00:00:00|2017-10-18 00:00:00|
|          8|  35182|             timoteo|        MG|2017-10

In [6]:
url = "jdbc:postgresql://localhost:5432/dbt"
properties = {
    "user": "dbt",
    "password": "dbt",
    "driver": "org.postgresql.Driver"
}

# Đọc dữ liệu từ PostgreSQL
df_orders = spark.read.jdbc(url=url, table="raw_layer.orders", properties=properties)

df_orders.show(20)

+--------------------+-------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|cust_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+-------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|     69|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|     17|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|     26|   delivered|     2018-08-08 08:38:49|2018-08-08 08:55:23|         2018-08-08 13

In [7]:
df_orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- cust_id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_carrier_date: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)
 |-- order_estimated_delivery_date: string (nullable = true)



In [8]:
df_orders.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/dbt") \
    .option("dbtable", "public.clm") \
    .option("user", "dbt") \
    .option("password", "dbt") \
    .save()